# Processes for Problem Solving
1. Recognizing a problem
2. Defining the problem
3. Structuring the problem
4. Analyzing the problem
5. Interpreting results & making a decision
6. Implementing the solution

# Data Preparation

In [ ]:
# Data Prep (use %>% to pipe)
count(columns)
spread(key=`col`, value=`col`) #change from long to wide form
pivot_wider()
pivot_longer()
summarize() #can use with min, max, sum, var, sd, first, n, n_distinct

#Types
as.factor(data$col, levels=c("val1", "val2"), ordered=T/F)

# Descriptive Analytics
**What Happened?**  
Using data to understand current and past business performance and make informed decisions.

## Visualisations in R
- **Categorical** data:
    - Bar Chart
    - Pie Chart
- **Continuous** data:
    - Histogram
- View **Relationships between Variables**:
    - Scatterplot
- Visualise **trends**:
    - Line chart

In [ ]:
# Using ggplot2
geom_bar(stat = 'identity', position='dodge/stack(default)') # Bar Charts
geom_histogram(fill='color', bins=30) # Histogram
geom_line() # Line Charts
geom_point() # Scatterplots

## Labels
labs(title ='', x ='', y ='', fill = '')
## For displaying tables
kable(data, caption='')

## For Multiple plots
library(gridExtra)
require(gridExtra)
grid.arrange(plot1, plot2, ncol=2, nrow=1)

### Tables
- Frequency Table: displays number of observations in **each group**
- Cumulative Relative Frequency: **proportion of total** observations that fall at or below the upper limit of each group
- Contigency Table: displays number of observations in **each category**

In [ ]:
## Pareto Analysis
pareto_data <- PCB_data %>% 
  select(Amount) %>% 
  arrange(desc(Amount)) %>% 
  mutate(Percentage = Amount/sum(Amount),  Cumulative = cumsum(Percentage)) 
### find first row that has 80% cumulative
pareto_row <- min(which(pareto_data$Cumulative > 0.8))
pareto_row / nrow(pareto_data)

## Contingency Tables
data.spread <- data %>% count(col, col) %>% spread(key=col, value=n)
kable(data.spread, caption="Contigency Table for ...")

## Interactive Contingency Table
library(rpivotTable)
rpivotTable(data, rows=c('header'), cols=c('header'))

## Probability Distributions & Data Modelling
- Classical definition: probabilities can be deduced from theoretical arguments
- Relative frequency definition: probabilities are based on empirical data
- Subjective definition: probabilities are based on judgment and experience

### Descriptive Statistics
- Kurtosis: +ve $\rightarrow$ distribution is peaked with less dispersion
- Coefficient of Skewness: +ve and >1 $\rightarrow$ high-degree of right skewness

In [ ]:
pnorm(1000, 750, 100) ## to get P(X < 1000) with X ~ N(750, 100^2)
pnorm(1000, 750, 100, lower.tail = FALSE) == 1 - pnorm(1000, 750, 100) ## to get P(X > 1000)
pnorm(1000, 750, 100) - pnorm(800, 750, 100) ## to get P(800 < X < 1000)

# Generate Descriptive Statistics
require(psych)
describe(col, data)
describeBy(data$col1, group=data$col2, mat=TRUE)

### Post-hoc Tests: Goodness of Fit
- Kolmogorow-Smirnov (works well for small samples and only for non-parametric data)
- Chi-square (needs at least 50 data points)
- Anderson-Darling (puts more weight on the differences between the tails of the distributions)
- Shapiro-Wilk Test (test data against normal distribution, check normality)

In [ ]:
# Kolmogorow-Smirnov test
# Chi-square test
# Anderson Darling test
# Shapiro-Wilk test
shapiro.test(data) # if W close to 1, p-value > 0.05 => implies data does not deviate from normality

# Density plot
plot(density(data))

# QQ plot
qqnorm(data)
qqline(data, col = 2)

# Boxplot (identify outliers)
geom_boxplot()

## Hypothesis Testing
Null Hypothesis $H_0$ and Alternative Hypothesis $H_1$
- Type 1 Error $\alpha = P(\text{rejecting } H_0 | H_0 \text{ is True})$
- Type 2 Error $\beta= P(\text{not rejecting } H_0 | H_0 \text{ is False})$

### Power of Test
Power of Test = $1 - \beta$ $\Rightarrow$ Probability of not commiting a Type 2 error
- Sensitive to Sample size: small sample sizes $\rightarrow$ low power

### One-Sample Hypothesis Tests
When testing for means, use **z-test** if variance $\sigma^2$ known. Use **t-test** if variance $\sigma^2$ unknown.
$$
z = \frac{\bar{x} - \mu_0}{\sigma / \sqrt{n}}\\
t = \frac{\bar{x} - \mu_0}{s / \sqrt{n}}
$$

Get p-value of $H_0$ = `pt(t-value, df=sample_size-1, lower.tail=TRUE/FALSE)`  
Reject $H_0$ if p-value < 0.05

To test for Proportion:  
$\Rightarrow$ Compute z-statistic for proportion  
$$
z = \frac{\hat{p} - \pi_0}{\sqrt{\pi_0(1-\pi_0)/n}}
$$
$\Rightarrow$ Compute critical value: `qnorm(0.05)`  
$\Rightarrow$ Check if z-statistic < critical value  
If z-statistic is **less than** critical value, then reject $H_0$


### Two-Sample Hypothesis Tests
If testing differences in **Mean**:
- If population variances $\sigma^2$ unknown,  
`t.test(outcome ~ group, alternative="greater/two.sided/less", data)`  
`t.test(group1, group2)`
- If variances known to be equal, set `var.equal=TRUE`  
- For Paired test, set `paired=TRUE`

If testing differences in **Variance**:
- `var.test(y~x)`

## Test for Equality of Variances
F-test: `var.test(outcome ~ group)`  
Other Tests: (if p-value < 0.05, no equal variance)
- Bartlett's Test `bartlett.test()`
    - For normally distributed data where there are 2 or more samples
- Levene's Test `leveneTest() in library(car)`
    - Alternative to Bartlett's test, less sensitive to departures from normality
- Fligner-Killeen Test `fligner.test()`
    - Non-parametric test that is very robust to departures from normality

### ANOVA
Compare means of two or more population groups.  
`aov_data <- aov(outcome~group, data)`  (group variable should be a factor)  
`summary(aov_data)`

Assumptions:
1. Independence $\rightarrow$ Randomly and independently obtained (validated if random samples were chosen)
2. Normality $\rightarrow$ Normally distributed (ANOVA is fairly robust to departures from normality)
3. Homogeneity $\rightarrow$ Have equal variances (okay if sample sizes are equal)

Can use Welch ANOVA test `welch_anova_test(data, variable~group)` if variances unequal

Post-Hoc tests: (Further Analysis)
- One-way ANOVA $\rightarrow$ use `TukeyHSD(aov_data)`
- Welch's ANOVA $\rightarrow$ use `games_howell_test(data, variable~group)`
- Control with familywise error rate

## Sampling and Estimation
### Confidence Interval
$$
\text{CI for Mean with Known StDev} = \bar{x} \pm z_{\alpha/2}\left(\frac{\sigma}{\sqrt{n}}\right)\\
\text{CI for Mean with Unknown StDev} = \bar{x} \pm t_{\alpha/2, n-1}\left(\frac{s}{\sqrt{n}}\right) \\
\text{CI for Mean for Proportion} = \hat{p} \pm z_{\alpha/2}\sqrt{\frac{\hat{p}(1-\hat{p})}{n}}
$$

$z_{\alpha/2}$ = `qnorm(1-alpha/2)`  
$t_{\alpha/2, n-1}$ = `qt((1-alpha/2), df=n-1)`

### Prediction Interval
(Need to **check for normality** first)
$$
\bar{x} \pm t_{\alpha/2, n-1}\left(s\sqrt{1 + \frac{1}{n}}\right)
$$

To reduce the sampling error to be within an error of $\pm E$
$$
\text{Sample size for Mean} = n \geq (z_{\alpha/2})^2\frac{\sigma^2}{E^2} \\
\text{Sample size of Proportion} = n \geq (z_{\alpha/2})^2\frac{\pi(1-\pi)}{E^2}
$$

Set $\pi = 0.5$ if no information is given about the population

In [ ]:
# CI for Mean (95% CI)
upperCI <- mean(data$Value) - qt(0.025, df=nrow(data)-1)*sd(data$Value)/sqrt(nrow(data))
lowerCI <- mean(data$Value) + qt(0.025, df=nrow(data)-1)*sd(data$Value)/sqrt(nrow(data))

# CI for Proportion (95% CI)
book <- data %>% filter(Product == "Book")
bk50 <- book %>% filter(Amount > 50)
propbk50 <- nrow(bk50) / nrow(book)
upperCI <- propbk50 - qnorm(0.025) * sqrt(propbk50*(1-propbk50)/nrow(book))
lowerCI <- propbk50 + qnorm(0.025) * sqrt(propbk50*(1-propbk50)/nrow(book))

# 95% Prediction Interval (Need to check for Normality)
mean.val <- mean(data$Value)
sd.val <- sd(data$Value)

upperPI <- mean.val + (qt(0.975, df=nrow(data)-1))*sd.val*sqrt(1+1/nrow(data))
lowerPI <- mean.val - (qt(0.975, df=nrow(data)-1))*sd.val*sqrt(1+1/nrow(data))

### Test for Normality
- Density Plot
- Histogram
- QQ plot
- Shapiro-Wilk Test

In [ ]:
# Density Plot
ggplot(data, aes(x=col)) +
    geom_density()

# QQ plot
qqnorm(data$col)
qqline(data$col)

# Shapiro-Wilk test
# if W close to 1, p-value > 0.05 => implies data does not deviate from normality
shapiro.test(data$col)

### Convert Data to Normal
Transform data using `transformTukey`, to get normally distributed data  
Need to convert results back for interpretation

In [ ]:
#Convert to Normal
library(rcompanion)
transformTukey(data$col) #note the lambda output

#Revert data back
upperResults <- transformedUpper ^ (1/lambda)
lowerResults <- transformedLower ^ (1/lambda)

# Predictive Analytics
**What will happen?**  
Predict future by examining historical data, detecting patterns or relationships in these data, then extrapolating these relationships forward in time.

## Linear / Logistic Regression

Steps to run prior to fitting a Regression Model:
1. Obtain descriptive statistics on all variables
    - Visualize key variables $\rightarrow$ using histograms, bar charts, line charts, scatterplots
    - Detect any outliers
2. Deal with legitmate outliers in Step 1
3. Check for any intercorrelations between independent variables
    - Generate correlation matrix
    - Remove/reduce multicollinearity by centered data, standardizing the independent variables, removing the variables

### Linear Regression / Ordinary Least Squares (OLS)
Minimises sum of squares of residuals
$$
Y = b_0 + b_1X_1 + b_2X_2 + b_3X_3 + \dots
$$
Where:
- $b_0$: Mean value of Y when X is zero
- $b_1$: A one unit change in $X_1$ results in a $b_1$ unit change in Y, when all other variables are held constant.
- $b_2$: A one unit change in $X_2$ results in a $b_2$ unit change in Y, when all other variables are held constant.
- $b_3$ (Assuming $X_3$ is categorical): Y is $b_3$ higher when $X_3$ is true, as compared to the reference group
- R-squared: measures the proportion of the variance explained by the model
- p-value: shows the significance of the regressions model

In [ ]:
# Linear Model 
fit1 <- lm(y~x1 + x2 + ..., df1)
summary(fit1)
predict(fit1)
residuals(fit1)

### Logistic Regression 
Predicting the log-odds of an event occuring
$$
logit(p) = log \frac{p}{1-p} = b_0 + b_1X_1 + b_2X_2
$$
Where:
- $b_0$: Log-odds when $X_1$ and $X_2$ are both zero
- $b_1$: Expected increase in log-odds of event when $X_1$ becomes 1, holding $X_2$ constant
- $b_2$: Expected increase in log-odds of event per unit-increase of $X_2$, holding $X_1$ constant / or multiples the odds by $exp(b_2)$
- Predicted odds = $exp(b_0 + b_1X_1 + b_2X_2)$

In [ ]:
# Logistic Regression
fit_log1 <- glm(y ~ x1 + x2 + ..., family="binomial", dataset)
predict(fit_log1, newdata)

## Time Series Forecasting
- Trends: Gradual upward or downward movement of time series over time
- Seasonality: Effect that occurs/repeats at a fixed interval
- Cyclical Effects: Longer-term effects that do not have a fixed interval/length
- Stationary: Statistical properties (mean, variance) of the time-series does not change over time

**Moving Average model:**
$$
Window_t = \frac{1}{n}\left(Y_t + Y_{t-1} + \dots + Y_{t-(n-1)}\right)
$$

**Exponential Smoothing model:**  
$\Rightarrow$ Gives more weight to recent values & uses all past data
$$
\hat{Y}_{t+1} = \alpha Y_t + (1-\alpha)\hat{Y}_t
$$

### Regression-based Forecasting
- Leading variable: $X$ is leading variable if changes in $X_t$ precedes changes in $Y_t$
- Lagging variable: $Y$ is lagging variable if it follows movements of other variables
- Coincident: both tend to co-vary at the same time

Interaction terms: Tests if the effect of one IV on the DV changes depending on the value of another IV
`lm(y ~ m*x, df) == lm(y ~ x + m + m*x, df)`  
Coefficient interpretation: Difference between the slopes of the groups

### Model Selection
**ANOVA:** Use ANOVA to compare if explanatory power of full model is significantly better than the restricted model.  
`anova(m_restricted, m_full)`

**Forward/Backwards Stepwise Regression:**  
`step(m_full, direction = "forward/backward/both")`

In [ ]:
# Moving Average model
library(TTR)
df$sma2 = SMA(df$y, n=2) # taking window size of 2
df$sma_predict = dplyr::lag(df$sma2, 1) # lagging by 1

# Exponential Smoothing model
# Single (no trend/seasonality)
HoltWinters(x, beta=FALSE, gamma=FALSE)

# Double (Trend & no seasonality)
HoltWinters(x, gamma=FALSE)

# Triple (Trend & seasonality)
HoltWinters(x)
predict(fit, n.ahead = k)

# Data Prep for Timeseries
library(tsbox)
ts_df(timeseries) #convert from Timeseries object to Dataframe
ts_ts(dataframe) #convert from Dataframe object to Timeseries

## Data Mining
### Data Exploration
Use a Pair Plot to visualise all the variables distribution and their correlation coefficients:

In [ ]:
library(psych)
pairs.panels(data, lm=TRUE)

### Data Reduction (Dimensionality Reduction)
**Principal Component Analysis (PCA)**  
$\Rightarrow$ Finds a set of features ("Principal Components") that explains the most variance in the data  
`pca <- prcomp(df, center=TRUE, scale=TRUE)`  
`summary(pca)`  
$\Rightarrow$ Important to have `scale=T` to standardize all the variables before running PCA

**k-means Clustering**  
`kmeans(df, num_of_clusters, nstart=n)`  
$\Rightarrow$ `nstart` argument, R will repeat kmeans with `n` different initialisations and return the best one

In [ ]:
# PCA
pca <- prcomp(df, center=T, scale=T)
summary(pca) # examining the output
pca$rotation[, 1:2] # examining loadings (distribution) on first 2 PCs
df$pc1 <- pca$x[, "PC1"] # extracting first PC
lm(col ~ pc1, df) # using the PC in linear model

## Plotting PCA
library(devtools)
install_github("vqv/ggbiplot")
library(ggbiplot)
ggbiplot(pca)

In [ ]:
# k-means Clustering
km_obj <- kmeans(df, num_of_clusters, nstart=10)
## to find optimal number of clusters (identify Elbow point)
wss <- rep(NA, 10)
for (k in c(1:10)) {
    wss[k] <- kmeans(df, k, nstart=10)$tot.withinss
}
plot(wss, type="b", xlab="Number of clusters", ylab="Total within-cluster sum of squares")

## Plotting the Clusters
fviz_cluster(km_obj, df)
## Get the centers of each cluster
km_obj$centers

# Regularisation
## alpha=0 (Lasso) / 1 (Ridge)
df_X = as.matrix(df[, X_COLS])
df_Y = as.matrix(df[, Y_COL])
lm_reg1 <- cv.glmnet(df_train_X, df_train_Y, alpha=0/1, family="gaussian")
coef(lm_reg1, s="lambda.min") # returns coeffs at the lambda that gives minimum MSE

### Data Classification  
$$
\text{Recall} = \frac{\text{True Positive}}{\text{True Positive + False Negative}}\\
\text{Precision} = \frac{\text{True Positive}}{\text{True Positive + False Positive}}\\
\text{F1-Score} = \frac{2 * \text{Precision} * \text{Recall}}{\text{Precision + Recall}}\\
$$

In [ ]:
# Confusion Matrix
table(data$col1, data$col2)

# Prescriptive Analytics
**How to make it happen?**  
Identify the best alternatives to minimize or maximize some objective.

## Linear Optimization (LP Relaxation)
Steps:  
1. Identify the Objective Function & Decision Variables
2. Identify the Constraints
3. Write down the Optimisation model
4. Use R to solve
5. Conduct Sensitivity Analysis
6. Interpret Results. Write a Recommendation.

### Sensitivity Analysis
**On Objective Function Coefficients**  
Shows the upper/lower bounds for the coefficients of variables, whereby the solution will stay optimal

**On Constraint Values**  
Shadow prices $\rightarrow$ how will increasing/decreasing constraint values affect the optimal solution

## Integer Optimisation
Constraint that the solution has to be an integer

**Logical Constraints:**
- If A, then B $\Rightarrow$ $B - A \geq 0 $
- If not A, then B $\Rightarrow$ $A + B \geq 1 $
- If A, then not B $\Rightarrow$ $A + B \leq 1 $
- At most one of A and B $\Rightarrow$ $A + B \leq 1 $
- If A, then B and C $\Rightarrow$ $B + C - 2A \geq 0 $
- If A and B, then C $\Rightarrow$ $A + B - C \leq 1 $

In [ ]:
#Linear/Integer Optimisation
library(lpSolve)
objective.fn <- c(2, 3, 5, 6, 8) # Profit = 2*X_1 + 3*X_2 + 5*X_3 + 6*X_4 + 8*X_5
const.mat <- matrix(c(rep(0,0), 1, -1, rep(0, 3), #Constraint1
                     rep(0,1), 1, -1, rep(0, 2), #Constraint2
                     rep(0,2), 1, -1, rep(0, 1) #Constraint3
                     ), ncol= 5, byrow=TRUE)
single_contraint <- (rep(0, 5), 1, -1, rep(0, 3)) #to write sparse constraints quickly
const.dir <- c("<=", "=", rep(">=", 3)) # ">=, =, <="
const.rhs <- c(100, 200, rep(50, 3)) # Right-side value of constraint
# lpSolve assumes non-negativity constraints -> no need to explicitly declare

#Solve Linear Optimisation
lp.soln <- lp("max/min", objective.fn, const.mat, const.dir, const.rhs, compute.sens=TRUE)
lp.soln # optimal objective function value
lp.soln$solution # decision variables values

##Sensitivity Analysis
### Bounds before the optimal solution will change
lp.soln$sens.coef.to # upper bounds on the coefficients 
lp.soln$sens.coef.from # lower bounds on the coefficients

##Shadow Prices
### Shows the increase/decrease in final objective value if constraint is increased by one unit
### Last two values shown are for the non-negativity constraints
lp.soln$duals

#Solve Integer Optimisation
int.soln <- lp("max/min", objective.fn, const.mat, const.dir, const.rhs, 
   int.vec = c(1,2,3), #define which variables have to be integers
   binary.vec = c(4:6), #define which variables are binary (0 or 1)
   compute.sens=FALSE) #sensitivity analysis doesnt work for integer constraint
int.soln
int.soln$solution